## 한국어 LLM 로컬 환경 테스트

- 먼저 base model 을 하나 선택해서 local mode로 테스트를 진행 해 봅니다.
- 모델 예시 : https://huggingface.co/LDCC/LDCC-SOLAR-10.7B

### 추천 설치 유틸리티

```
# Install tools
yum -y install ncurses-devel tmux htop libdrm-devel

# Install nvtop
git clone https://github.com/Syllo/nvtop.git
mkdir -p nvtop/build && cd nvtop/build
cmake .. -DNVIDIA_SUPPORT=ON -DAMDGPU_SUPPORT=OFF
make
make install
```


### Tested version

```
bitsandbytes              0.43.0
datasets                  2.18.0
peft                      0.9.0
transformers              4.38.2
```

먼저 필요한 패키지들을 설치합니다.
- 아래 명령어는 최신 버전 패키지를 설치하기 때문에, 상황에 따라 위에 명시된 버전으로 설치하는 것을 추천합니다.

In [ ]:
!pip install -q transformers peft bitsandbytes datasets

In [ ]:
!pip list | grep 'peft\|transformers\|bitsandbytes\|datasets'

### 모델 로딩 및 테스트

- LLM 활용 시 가장 많이 사용되는 [HF transformers](https://huggingface.co/docs/transformers/index) 를 사용합니다.
- [bitsandbytes](https://github.com/TimDettmers/bitsandbytes) 를 사용해서 Quantization을 합니다.
- 여기서는 4bit Quantization을 하여 메모리 사용량을 줄입니다.
- 모델을 로컬에 다운로드 받고 이것을 로드하여 테스트합니다.


In [ ]:
import sys
import os
import torch
import transformers

In [ ]:
from transformers import BitsAndBytesConfig
quant_4bit = True
quant_8bit = False

if quant_4bit:
    nf4_config = BitsAndBytesConfig(
       load_in_4bit=True,
       bnb_4bit_quant_type="nf4",
       bnb_4bit_use_double_quant=True,
       bnb_4bit_compute_dtype=torch.bfloat16
    )
else:
    nf4_config = None

In [ ]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "LDCC/LDCC-SOLAR-10.7B"
revision = "v1.2"

# create model dir
model_name = HF_MODEL_ID.split("/")[-1].replace('.', '-')
model_tar_dir = Path(f"/home/ec2-user/SageMaker/models/{model_name}")

In [ ]:
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model", "*.py", "*.safetensors"]

model_download_path = snapshot_download(
    repo_id=HF_MODEL_ID,
    cache_dir=model_tar_dir,
    allow_patterns=allow_patterns,
    revision=revision
)

In [ ]:
print(model_download_path)
print(model_tar_dir)

In [ ]:
# Example: When using specific path
# base_model_path = "base_model/models--LDCC--LDCC-SOLAR-10.7B/snapshots/1055563879363d9ee2fba1d9fd1628eca6bcbb4e"
# model_download_path = "merged_model"
# model_download_path = "model_from_sagemaker"

### Inference 테스트

- 모델을 로드하여 테스트합니다.
- base model 이 instruction tuning이 된 경우 어떤 포맷을 활용해서 학습이 되어있는지를 확인할 필요가 있습니다.
- 예를 들어 solar instruction 같은 경우 [여기](https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0#conducting-single-turn-conversation) 내용과 같은 포맷을 활용합니다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
%%time

device_map = "auto"

tokenizer = AutoTokenizer.from_pretrained(model_download_path)

model = AutoModelForCausalLM.from_pretrained(
    model_download_path,
    load_in_8bit=True if quant_8bit else False,
    torch_dtype=torch.float16,
    device_map=device_map,
    quantization_config=nf4_config,
)

In [ ]:
# prompt = "초록색 가래가 계속 나오고 기침이 나오는데, 큰 병은 아닐까요?"
# prompt = "항문압 측정 검사에서 항문 압력이 증가하는 경우는 어떤 경우일까요?"
# prompt = "55세 남자가 위암으로 위절제술을 받았다. 수술 후 3일째 혈색소가 6.7 g/dL,로 감소하여 농축적혈구 수혈을 시작하였다. 수혈 도중 갑자기 오한과 발열이 발생하였으며, 복부 피부절개 부위에서 혈성 삼출물이 보이고 수혈 주사부위에 통증이 생겼다. 혈압 100/60 mmHg, 맥박 102회/분, 호흡 24회/분, 체온 38.2도 이다. 처치는?"
# prompt = "감염병 유행 역학조사를 수행하면서 인구학적 특성별, 발생 시기별로 환자발생 점지도(spot map) 를 작성하였다. 이를 통해 알 수 있는 것은?"
# prompt = "42세 남자가 1개월 전부터 숨이 많이 찬다며 병원에 왔다. 4년 전부터 숨이 찼다고 한다. 1주 전부터는 기침도 하고, 밤에 잘 때 쌕쌕거리는 소리도 난다고 한다. 20갑 ∙ 년의 흡연자이다. 혈압 120/70 mmHg, 맥박 88회/분, 호흡 20회/분, 체온 36.4℃이다. 가슴 청진에서 호흡음은 정상이다. 가슴 X선사진은 정상이다. 폐기능검사 결과는 다음과 같다. 다음 검사는?강제폐활량: 3.0 L (예측치의 92%) 1초간 강제날숨량: 2.7 L (예측치의 90%)"
prompt = "1개월 남아가 12시간 전부터 피부색이 창백하고 얼룩덜룩하게 변하여 병원에 왔다. 하루 전부터 많이 보채면서 한 번에 분유를 30 mL도 못 먹었지만 배가 불러보였다고 한다. 어딘지 평소와 다르게 보였고, 집에서 측정한 체온은 35.9℃부터 37.7℃까지 오르내렸다. 임신나이 39주, 출생체중 3,300 g, 질분만으로 태어나서, 아프기 전에는 잘 지내왔었다. 혈압 65/45 mmHg, 맥박 155회/분, 호흡 60회/분, 체온 39.2℃이다. 피부색은 창백하고, 진찰하는 동안에도 많이 처져서 잘 울지 않는다. 검사는?" 
# prompt = "우리나라는 환자의 수도권 집중 방지와 지방의 진료 역량 강화를 위하여 수도권을 제외한 시도에 권역 단위로 전문질환센터를 지정하여 지원하고 있다. 권역 전문질환센터의 설립 취지를 반영하는 병원 관리 평가지표는?"
# prompt = "65세 이상 노인에게서 인플루엔자 예방접종이 폐렴으로 인한 입원율을 낮추는 효과가 있는지 확인하려고 한다. 일정기간 폐렴으로 인한 입원 환자 수는 접종군 4,000명 중 20명, 비접종군 2,000명 중 40명이다. 인플루엔자 예방접종의 폐렴 입원에 대한 예방효과는?"
# prompt = "펜션에서 투숙객 3명이 혼수상태로 발견되었다. 특별한 외상은 관찰되지 않았으며, 혈액검사에서 일산화탄소헤모글로빈(COHb) 은 40% 이상이었다. 치료는?"
# prompt = "미생물연구실에서 실험을 하던 25세 여자가 눈의 충혈, 이물감, 시림, 통증으로 병원에 왔다. 이 여자는 보안경을 착용하지 않은 채 무균실험대 위에서 살균등을 켜고 실험을 하였다. 노출이 의심되는 방사선은?"
# prompt = "42세 남자가 화를 못 참겠다며 병원에 왔다. 1개월 전에 골육종으로 오른쪽 다리를 절단해야 한다는 진단을 받았다. 상급종합병원에서 다시 한 번 검사를 받았으나 결과는 같았다. 왜 이런 일이 생긴 건지 믿을 수가 없다며 화를 내다가도 아직 7개월밖에 안 된 아들을 생각하면 갑자기 눈물이 나와 멈출 수가 없었다. 직장에서는 사소한 일에도 화가 치밀어 동료들과 다툼이 잦아졌고 업무에도 실수가 많아져 사람이 갑자기 변했다는 소리를 들었다고 한다. 밤에는 미래에 대한 걱정으로 잠을 거의 자지 못했고 식욕이 없어 하루에 한 끼를 겨우 먹었다. 치료는?"

In [ ]:
instruction = """
당신은 똑똑한 의사입니다. 질문에 대해서 최대한 자세하게 어떤 병인지, 치료법은 무엇인지 환자에게 설명하듯 친절하게 설명해 주세요.
"""


In [ ]:
# Normal format
text = f"""
{instruction}

{prompt}
"""

# # IFT format
# text = f"""
# <s> ### User:
# {instruction}

# {prompt}

# ### Assistant:
# """

print(text)

In [ ]:
params = {
    "max_new_tokens": 512,
    "temperature": 0.1,
    "top_p": 0.1,
    "do_sample": True
}

In [ ]:
%%time

model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

generated_ids = model.generate(
    model_inputs.input_ids,
    **params
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)

In [ ]:
%store model_download_path